# Blatt 3
## Aufgabe 8: Importance Sampling

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

__a)__ Die Planck-Verteilung:

In [ ]:
def Planck(x, N = 15 / np.pi**4): 
    return N * x**3 / (np.exp(x) - 1)

Bestimme zunächst numerisch das Maximum:

In [ ]:
from scipy.optimize import brentq 

N = 15 / np.pi**4

def diff_Planck(x):
    return N * (3 * x**2 * (np.exp(x) - 1) - x**3 * np.exp(x)) / (np.exp(x) - 1)**2 

xmax = brentq(diff_Planck, 1, 4)
ymax = Planck(xmax)

Erzeuge 2 gleichverteilte Datensätze der Länge 5e4, also insgesamt 25e8 Zahlen:

In [ ]:
xcut = 20
totnumber = 500000
uniformx = np.random.uniform(0, xcut, totnumber)
uniformy = np.random.uniform(0, ymax, totnumber)

In [ ]:
def Rejection_sampling(u1, u2, f, length = 100000):
    sample = u1[u2 <= f(u1)]
    assert len(sample) >= length
    return sample[np.random.randint(0, len(sample), length)]
    

In [ ]:
planck_sample_a = Rejection_sampling(uniformx, uniformy, Planck)

__b)__ Bestimme zunächst den Schnittpunkt der Majoranten x_s:

In [ ]:
def func(x, N = 15 / np.pi**4, xmax = xmax):
    return ymax - 200 * N * x**(-0.1) * np.exp(-x**(0.9)) 

x_s = brentq(func, 1, 6)
x_s

In [ ]:
def func_g(x, x_s = x_s, N = 15 / np.pi**4):
    y = np.zeros(len(x))
    y[x <= x_s] = Planck(xmax)
    y[x > x_s] = 200 * N * x[x >= x_s]**(-0.1) * np.exp(-x[x >= x_s]**(0.9))
    return y

In [ ]:
def inv_G(y, x_s = x_s, xcut = xcut):
    return (- np.log(np.exp(-x_s**(0.9)) 
                     + y * (np.exp(-xcut**(0.9)) 
                            - np.exp(-x_s**(0.9))) ))**(10 / 9)

norm = x_s * ymax + 2000 / 9 * N * (np.exp(-x_s**(9/10)) - np.exp(-xcut**(9/10)) )
                                                                

totnumber = 200000
part_uniform = int(x_s * ymax / norm * totnumber)

uniformx = np.random.uniform(0, 1, totnumber - part_uniform)
uniformx2 = np.random.uniform(0, x_s, part_uniform)
uniformy = np.random.uniform(0, 1, totnumber)
uniformx = np.concatenate([uniformx2, inv_G(uniformx)])
len(uniformx)

In [ ]:
plt.clf()
xplot = np.linspace(0.01, xcut, 1000)
plt.plot(xplot, func_g(xplot))
plt.plot(xplot, Planck(xplot))
plt.scatter(uniformx, func_g(uniformx) * uniformy, marker = '.', alpha = 0.1)
plt.show()

In [ ]:
planck_sample_b = Rejection_sampling(uniformx, 
                                     func_g(uniformx) * uniformy, 
                                     Planck)

In [ ]:
fig = plt.figure(figsize = (20, 10))

fig.add_subplot(121)
plt.hist(planck_sample_a, bins = 30, histtype = 'step',
         density = True, 
         label = 'Aufgabenteil a)', 
         linewidth = 3)
plt.plot(xplot, Planck(xplot), 
         color = 'r', linewidth = 3)
plt.legend()

fig.add_subplot(122)
plt.hist(planck_sample_b, bins = 30, histtype = 'step',
         density = True, 
         label = 'Aufgabenteil b)', 
         linewidth = 3)
plt.plot(xplot, Planck(xplot), 
         color = 'r', linewidth = 3)
plt.legend()

plt.show()